# TensorBoard: Convolution Activation Map Visualization / Embedding

이번 튜토리얼에서는 공식 홈페이지에서 제공하는 튜토리얼인 [Tensorboard: Embedding Visualization](https://www.tensorflow.org/how_tos/embedding_viz/)를 참고하여 mnist data (784x1 짜리 vector)를 visualization시켜 보도록 하겠습니다.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets 
import tensorflow as tf
import numpy as np
import os

### Load MNIST dataset

In [2]:
mnist = input_data.read_data_sets(train_dir='mnist', reshape=False)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


### Define the Model

In [3]:
class convnet(object):
    def __init__(self, num_class=10, learning_rate=0.001):
        self.input = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name='input_images')
        self.target = tf.placeholder(tf.int64, shape=[None], name='target_indices')
        self.num_class = num_class
        self.learning_rate = learning_rate
    
        # build model
        with slim.arg_scope([slim.conv2d], padding='SAME', activation_fn=tf.nn.relu,
                            weights_initializer=tf.contrib.layers.xavier_initializer()):
            
            self.conv1 = slim.conv2d(self.input, 16, [5, 5], scope='conv1') 
            self.pool1 = slim.max_pool2d(self.conv1, [2, 2], scope='pool1')  # (batch_size, 14, 14, 16)            
            
            self.conv2 = slim.conv2d(self.pool1, 32, [5, 5], scope='conv2')
            self.pool2 = slim.max_pool2d(self.conv2, [2, 2], scope='pool2')  # (batch_size, 7, 7, 32)
            
            self.conv3 = slim.conv2d(self.pool2, 64, [5, 5], scope='conv3')
            self.pool3 = slim.max_pool2d(self.conv3, [2, 2], scope='pool3')  # (batch_size, 3, 3, 64)
            
            self.fc = slim.fully_connected(slim.flatten(self.pool3), 128, scope='fc')
            self.out = slim.fully_connected(self.fc, 10, activation_fn=None, scope='out')
            
            
            self.pred = tf.argmax(self.out, 1)
            self.correct_pred = tf.equal(self.pred, self.target)
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))
            self.loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(self.out, self.target))
            self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)

In [4]:
model = convnet()

In [5]:
num_epoch = 10
batch_size = 100
num_iter_per_epoch = int(x_train.shape[0] / batch_size)

gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=gpu_config) 

tf.global_variables_initializer().run()
for e in range(num_epoch):
    avg_loss = 0.0
    avg_acc = 0.0
    for i in range(num_iter_per_epoch):
        x_batch = x_train[i*batch_size:(i+1)*batch_size]
        y_batch = y_train[i*batch_size:(i+1)*batch_size]
        
        feed_dict = {model.input: x_batch, model.target: y_batch}
        _, l, acc = sess.run([model.train_op, model.loss, model.accuracy], feed_dict=feed_dict)
        
        avg_loss += l / num_iter_per_epoch
        avg_acc += acc / num_iter_per_epoch
    print('Epoch [%d/%d], Loss: %.4f, Accuracy: %.4f' %(e+1, num_epoch, avg_loss, avg_acc))

Epoch [1/10], Loss: 0.2432, Accuracy: 0.9263
Epoch [2/10], Loss: 0.0635, Accuracy: 0.9800
Epoch [3/10], Loss: 0.0390, Accuracy: 0.9881
Epoch [4/10], Loss: 0.0280, Accuracy: 0.9914
Epoch [5/10], Loss: 0.0227, Accuracy: 0.9931
Epoch [6/10], Loss: 0.0173, Accuracy: 0.9950
Epoch [7/10], Loss: 0.0161, Accuracy: 0.9950
Epoch [8/10], Loss: 0.0149, Accuracy: 0.9954
Epoch [9/10], Loss: 0.0116, Accuracy: 0.9960
Epoch [10/10], Loss: 0.0119, Accuracy: 0.9959


### Define Embedding Variable

In [14]:
num_iter = int(x_test.shape[0] / batch_size)
features = np.zeros((x_test.shape[0], 128))

for i in range(num_iter):
    x_batch = x_test[i*batch_size:(i+1)*batch_size]
    y_batch = y_test[i*batch_size:(i+1)*batch_size]

    features[i*batch_size:(i+1)*batch_size] = sess.run(fetches=model.fc, 
                                                       feed_dict={model.input: x_batch, model.target: y_batch})

In [29]:
embedding_var = tf.Variable(features, name='embedding')

In [30]:
# make log directory if not exists
log_dir = 'logs/'
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

In [39]:
tf.global_variables_initializer().run()
saver = tf.train.Saver()
saver.save(sess, save_path=log_dir+'model.ckpt', global_step=None)

# (required) add embedding data
summary_writer = tf.summary.FileWriter(logdir=log_dir, graph=tf.get_default_graph())
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
embedding.sprite.image_path = 'mnist/mnist_10k_sprite.png'
# Specify the width and height of a single thumbnail.
embedding.sprite.single_image_dim.extend([28, 28])

In [40]:
# (optional) add meta data such as labels and sprite image 
metadata_path = 'logs/metadata.tsv'
with open(metadata_path, 'w') as f:
    for i, label in enumerate(mnist.test.labels):
        f.write('{}\n'.format(label))
embedding.metadata_path = metadata_path

In [41]:
# visualize embedding projector
projector.visualize_embeddings(summary_writer, config)